In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import time

In [2]:
master_artists = pd.read_csv("master_artists_list.csv")

In [3]:
master_artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube'],
      dtype='object')

In [12]:
df = pd.DataFrame(columns = ['artist_name','title','release_date','primary_type'])

In [13]:
for i in range(0,130,10):
    time.sleep(5)
    for artist in master_artists['artist_name'].values[i:i+10]:
        url = ""
        if (artist == 'Beyoncé'):
            url = "http://musicbrainz.org/ws/2/release/?query=artist:beyonce&fmt=json"
        elif (artist == 'AC/DC'):
            url = "http://musicbrainz.org/ws/2/release/?query=artist:ac%2fdc&fmt=json"
        elif (artist == 'MØ'):
            url = "http://musicbrainz.org/ws/2/release/?query=arid:3ab2ee23-1c2f-458c-9c8c-ca4844729120&fmt=json"
        else:
            artist_url = artist.lower().replace(" ", "%20")
            url = "http://musicbrainz.org/ws/2/release/?query=artist:" + artist_url+ "&fmt=json"
        contents = urllib.request.urlopen(url).read()
        channels = json.loads(contents)
        for releases in channels['releases']:
            features = []
            features.append(artist)
            if 'title' in releases:
                features.append(releases['title'])
            else:
                features.append(np.NAN)

            if 'date' in releases:
                features.append(releases['date'])
            else:
                features.append(np.NAN)

            if 'release-group' in releases:
                if 'primary-type' in releases['release-group']:
                    features.append(releases['release-group']['primary-type'])
                else:
                    features.append(np.NAN)
            else:
                features.append(np.NAN)
            df = df.append(pd.Series(features,index = df.columns),ignore_index=True)

In [16]:
df.count()

artist_name     3222
title           3222
release_date    2857
primary_type    3169
dtype: int64

In [18]:
df['release_date'].isnull().values.sum()

365

In [19]:
df = df.dropna(axis=0, subset=['release_date'])

In [20]:
df.count()

artist_name     2857
title           2857
release_date    2857
primary_type    2832
dtype: int64

In [21]:
df.dtypes

artist_name     object
title           object
release_date    object
primary_type    object
dtype: object

In [26]:
df['release_date'].str.len().unique()

array([ 4, 10,  7,  0], dtype=int64)

In [32]:
df = df[df['release_date'].str.len() != 0]

In [33]:
df.count()

artist_name     2796
title           2796
release_date    2796
primary_type    2775
dtype: int64

In [43]:
df[df['release_date'].str.len() == 4].release_date.describe()

count      885
unique      64
top       1996
freq        53
Name: release_date, dtype: object

In [46]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d %H:%M:%S')

In [50]:
df.groupby(['artist_name','title']).count()

release_date  \
artist_name  title                                                   
21 Savage    Savage                                             22   
             Savage Mode                                         1   
AC/DC        A Vulgar Display of Ultra Rare Tracks               1   
             All My Friends Are Gonna Be There Too               1   
             Black Ice                                           3   
             Bonfire Sampler                                     1   
             Dirty Deeds Done Dirt Cheap                         1   
             Dirty Deeds Done Dirt Cheap (live)                  1   
             Girls Got Rhythm                                    1   
             Highway to Hell                                     5   
             Live at Donington                                   1   
             Moneytalks                                          1   
             Plug Me In: Bonus CD                                1   
             Rare, Rarer, Rarities                               2   
             Thunderball Boogie                                  1   
             Ultimate Volts                                      1   
             Warning! High Voltage (Greatest Hits!)              1   
ASAP Ferg    Clumsy / Fergalicious                               1   
             Cool John Ferguson                                  1   
             Ferg Forever                                        1   
             Fergalicious                                        2   
             Fergean                                             2   
             Fergeteg                                            1   
             Fergus Sings the Blues                              1   
             Ferguslie Park                                      3   
             Harmaa Ferguson                                     1   
             Headliners:03: Fergie                               1   
             Jönnek a férgek                                     1   
             Maynard Ferguson                                    1   
             Maynard Ferguson Octet                              1   
...                                                            ...   
Zara Larsson Ain’t My Fault                                      1   
             Ain’t My Fault (J HUS & FRED VIP remix)             1   
             Ain’t My Fault (R3hab remix)                        1   
             Ain’t My Fault (remix)                              1   
             All the Time                                        1   
             Allow Me to Reintroduce Myself                      1   
             Bad Boys                                            1   
             Carry You Home (remixed)                            1   
             I Would Like                                        1   
             I Would Like (Gorgon City remix)                    1   
             I Would Like (R3hab remix)                          1   
             Introducing                                         1   
             Larsson                                             1   
             Livin La Vida Larsson                               1   
             Lush Life                                           1   
             Only You + Remixes                                  1   
             So Good                                             4   
             Spotify Singles                                     1   
             Uncover (the remixes)                               1   
             WOW                                                 1   
Zayn         BeFoUr                                              1   
             Fingers                                             1   
             Icarus Falls                                        4   
             LIKE I WOULD                                        1   
             LIKE I WOULD (The Remixes)                         

In [51]:
df.dtypes

artist_name             object
title                   object
release_date    datetime64[ns]
primary_type            object
dtype: object

In [54]:
df.sort_values(by=['artist_name','title','release_date'],inplace=True)

In [55]:
df.head()

,artist_name,title,release_date,primary_type
1459,21 Savage,Savage,1982-01-01,Album
1461,21 Savage,Savage,1982-01-01,Album
1454,21 Savage,Savage,1987-01-01,Album
1463,21 Savage,Savage,1987-01-01,Album
1466,21 Savage,Savage,1987-01-01,Album


In [57]:
df = df.groupby(['artist_name','title']).first()

In [62]:
df.reset_index(inplace=True)

In [63]:
df.head()

,artist_name,title,release_date,primary_type
0,21 Savage,Savage,1982-01-01,Album
1,21 Savage,Savage Mode,2016-07-15,Album
2,AC/DC,A Vulgar Display of Ultra Rare Tracks,2005-01-01,Album
3,AC/DC,All My Friends Are Gonna Be There Too,2006-07-21,Album
4,AC/DC,Black Ice,2008-10-20,Album


In [64]:
df.to_csv('musicbrainz_data.csv')